In [31]:
using BenchmarkTools

In [32]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [33]:
function Convolution_2d_pad(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_pad (generic function with 1 method)

In [34]:
function Convolution_2d(input, kernel; bias=0.)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [35]:
function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

function Convolution_2d_v2!(ret, sumret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

Convolution_2d_v2! (generic function with 1 method)

In [36]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d_pad(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [37]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d_pad(tmp_weights, rotate(tmp_gradient); padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [38]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [39]:
function Conv_backward_v4(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
            grad_weights[:, :, k, c] .+= tmp_weights
            tmp_weights .= 0
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v4 (generic function with 1 method)

In [40]:
function Conv_backward_v5(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v5 (generic function with 1 method)

In [41]:
function Conv_backward_v6(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          @views grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v6 (generic function with 1 method)

In [42]:
function Conv_backward_v7(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  sumret = zeros(Float32, output_height, output_width)
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d_v2!(tmp_weights, sumret, tmp_input, tmp_gradient)
          @views grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v7 (generic function with 1 method)

In [43]:
input = rand(28, 28, 1);
weights = rand(3, 3, 1, 6);
bias = rand(Float32, 6);
gradient = rand(26, 26, 6);

In [44]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 1196 samples with 1 evaluation.
 Range (min … max):  2.889 ms … 35.489 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):     4.222 ms              ┊ GC (median):    27.21%
 Time  (mean ± σ):   4.172 ms ±  1.621 ms  ┊ GC (mean ± σ):  18.55% ± 14.22%

   █       ▆▃                                                 
  ▅█▆▄▄▃▃▂▅██▇▇▅▄▃▃▃▂▂▂▁▂▂▁▂▁▂▂▂▁▁▁▁▁▂▂▁▂▂▁▁▂▁▂▂▁▁▁▂▂▁▂▁▂▂▂▂ ▃
  2.89 ms        Histogram: frequency by time        11.1 ms <

 Memory estimate: 25.49 MiB, allocs estimate: 4842.

In [45]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 1393 samples with 1 evaluation.
 Range (min … max):  2.754 ms …  10.249 ms  ┊ GC (min … max):  0.00% … 53.57%
 Time  (median):     2.955 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.581 ms ± 999.375 μs  ┊ GC (mean ± σ):  16.31% ± 18.18%

  ▃▇█▅▄▂▁▁▁                   ▁▅▄▄▃▂▁▂                         
  █████████▇▇▇▆▆▆▄▆▆▄▅▄▄▄▁▁▄▁▄███████████▇▇▆▆▅▄▄▄▅▄▁▅▁▄▁▁▁▅▁▅ █
  2.75 ms      Histogram: log(frequency) by time       6.5 ms <

 Memory estimate: 13.22 MiB, allocs estimate: 4827.

In [46]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  305.900 μs …   1.825 ms  ┊ GC (min … max): 0.00% … 78.25%
 Time  (median):     315.600 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   334.767 μs ± 116.240 μs  ┊ GC (mean ± σ):  3.53% ±  8.05%

  █▅▂▂▁                                                         ▁
  ██████▇▇▆▇█▆▆▆▆▆▆▆▆▅▄▄▅▃▁▁▃▁▃▃▃▁▁▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆ █
  306 μs        Histogram: log(frequency) by time       1.28 ms <

 Memory estimate: 544.08 KiB, allocs estimate: 4149.

In [47]:
@benchmark Conv_backward_v4(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  307.700 μs …  1.611 ms  ┊ GC (min … max): 0.00% … 50.33%
 Time  (median):     313.800 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   328.996 μs ± 77.544 μs  ┊ GC (mean ± σ):  1.80% ±  6.17%

  █▆▄▂▁                                                        ▁
  ████████▇▆▇▆▆▆▅▄▄▅▅▆▇▅▅▆▆▆▇▆▆▅▆▇▆▃▄▄▃▄▅▅▄▅▃▄▄▁▁▁▁▁▁▁▁▁▃▁▁▃▆▇ █
  308 μs        Histogram: log(frequency) by time       772 μs <

 Memory estimate: 423.42 KiB, allocs estimate: 4083.

In [48]:
@benchmark Conv_backward_v5(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  126.900 μs …  1.879 ms  ┊ GC (min … max): 0.00% … 91.05%
 Time  (median):     130.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   133.977 μs ± 49.088 μs  ┊ GC (mean ± σ):  1.05% ±  2.72%

  █▆▇▇▅▅▅▄▄▃▂▂▂▁▂▁▁▁                                           ▂
  ███████████████████▇█▇▇▇▇▆▆▇▆▆▆▅▅▄▅▆▆▅▄▃▃▆▄▅▄▄▅▅▃▃▃▁▄▄▁▄▁▁▄▃ █
  127 μs        Histogram: log(frequency) by time       193 μs <

 Memory estimate: 43.27 KiB, allocs estimate: 28.

In [49]:
@benchmark Conv_backward_v6(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  128.000 μs …  1.807 ms  ┊ GC (min … max): 0.00% … 90.71%
 Time  (median):     131.200 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   135.974 μs ± 48.436 μs  ┊ GC (mean ± σ):  1.00% ±  2.71%

  █▆█▅▅▅▄▃▂▂▁▁▁▁                                               ▂
  ███████████████████▇▆▇▆▇▇▆▅▅▆▄▅▅▅▅▄▄▅▃▅▅▅▄▄▄▂▃▂▄▅▂▃▄▄▄▄▄▃▄▃▄ █
  128 μs        Histogram: log(frequency) by time       212 μs <

 Memory estimate: 42.70 KiB, allocs estimate: 22.

In [50]:
@benchmark Conv_backward_v7(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  133.900 μs …  2.233 ms  ┊ GC (min … max): 0.00% … 92.61%
 Time  (median):     136.700 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   138.903 μs ± 30.510 μs  ┊ GC (mean ± σ):  0.29% ±  1.31%

  █▇▅▇█▇▅▄▄▅▄▃▂▂▂▁▁▁▁▁ ▁                                       ▂
  █████████████████████████▇██▇▇▇▆▆▆▅▆▆▆▅▆▅▆▅▃▄▄▅▅▅▅▄▃▄▄▃▅▄▅▄▄ █
  134 μs        Histogram: log(frequency) by time       175 μs <

 Memory estimate: 12.80 KiB, allocs estimate: 16.

In [51]:
Conv_backward_v4(input, weights, bias, gradient)

(Float32[0.8128891 2.9091341 … 3.4959154 1.6244532; 1.7896742 5.67576 … 5.367323 2.2714581; … ; 2.5396886 3.932515 … 3.4926639 0.986063; 1.2906618 1.6006047 … 2.1973867 0.78863555;;;], Float32[170.1107 170.64433 174.66379; 172.65263 174.50587 172.66826; 175.52145 175.46959 176.91377;;;; 171.09254 176.68437 169.5569; 177.13745 171.16148 175.49277; 172.89124 173.16795 174.92757;;;; 172.16434 169.14175 167.6081; 173.81488 169.10178 172.93605; 168.32927 168.30878 168.49715;;;; 170.10854 167.15915 166.1391; 170.58748 173.52782 173.94142; 173.79782 171.90703 170.21892;;;; 161.61462 163.38979 166.72002; 170.20479 166.30374 166.9147; 163.77975 168.74579 163.94263;;;; 166.20746 164.30222 159.97626; 167.78616 165.36838 166.24463; 168.38484 170.07202 167.06648], [342.34181983324476, 342.3148264043288, 336.72381260122995, 337.30530966477767, 325.6651151038503, 328.1599197854253])

In [52]:
Conv_backward_v6(input, weights, bias, gradient)

(Float32[0.81288904 2.9091341 … 3.4959154 1.6244532; 1.7896742 5.67576 … 5.3673234 2.2714584; … ; 2.5396883 3.932515 … 3.492664 0.98606294; 1.2906618 1.6006047 … 2.1973867 0.7886355;;;], Float32[170.1107 170.64433 174.66379; 172.65263 174.50587 172.66826; 175.52145 175.46959 176.91377;;;; 171.09254 176.68437 169.5569; 177.13745 171.16148 175.49277; 172.89124 173.16795 174.92757;;;; 172.16434 169.14175 167.6081; 173.81488 169.10178 172.93605; 168.32927 168.30878 168.49715;;;; 170.10854 167.15915 166.1391; 170.58748 173.52782 173.94142; 173.79782 171.90703 170.21892;;;; 161.61462 163.38979 166.72002; 170.20479 166.30374 166.9147; 163.77975 168.74579 163.94263;;;; 166.20746 164.30222 159.97626; 167.78616 165.36838 166.24463; 168.38484 170.07202 167.06648], [342.34181983324476, 342.3148264043288, 336.72381260122995, 337.30530966477767, 325.6651151038503, 328.1599197854253])

In [53]:
Conv_backward_v7(input, weights, bias, gradient)

(Float32[0.81288904 2.9091341 … 3.4959154 1.6244532; 1.7896742 5.67576 … 5.3673234 2.2714584; … ; 2.5396883 3.932515 … 3.492664 0.98606294; 1.2906618 1.6006047 … 2.1973867 0.7886355;;;], Float32[170.1107 170.64433 174.66379; 172.65263 174.50586 172.66827; 175.52144 175.4696 176.91376;;;; 171.09254 176.68437 169.55692; 177.13745 171.16148 175.49277; 172.89124 173.16797 174.92757;;;; 172.16434 169.14175 167.6081; 173.81487 169.10179 172.93605; 168.32927 168.30878 168.49712;;;; 170.10854 167.15912 166.13911; 170.58748 173.52782 173.94142; 173.79782 171.90703 170.21892;;;; 161.61462 163.38979 166.72002; 170.20479 166.30374 166.9147; 163.77975 168.74579 163.94261;;;; 166.20744 164.30222 159.97624; 167.78618 165.36838 166.24461; 168.38486 170.07202 167.06647], [342.34181983324476, 342.3148264043288, 336.72381260122995, 337.30530966477767, 325.6651151038503, 328.1599197854253])